In [1]:
import os

In [2]:
pwd

'c:\\Users\\gowtham.kaliraj\\Desktop\\ML projects\\Text-summarizer-project\\research'

In [7]:
os.chdir("../")


In [8]:
pwd

'c:\\Users\\gowtham.kaliraj\\Desktop\\ML projects\\Text-summarizer-project'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filename = CONFIG_FILE_PATH,
        params_filename = PARAMS_FILE_PATH,
    ) -> None:
        self.config = read_yaml(config_filename)
        self.params = read_yaml(params_filename)
    
        create_directories([self.config.artifacts_root], True)
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_url
        )       
        
        return data_ingestion_config
        

In [15]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-17 20:31:59,229: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-17 20:31:59,231: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-17 20:31:59,232: INFO: common: Created directory at: artifacts]


TypeError: isinstance() arg 2 must be a type or tuple of types